In [1]:
import torch,torchvision,os,pyhessian,time
import torchvision.transforms as transforms
import numpy as np
import pyhessian
from utils.util import get_loader,evaluate
from utils.layer import qConv2d,qLinear
from utils.train import QAVAT_train
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True).cuda()

Using cache found in /home/zihao/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [2]:
from copy import deepcopy
int4model = deepcopy(model)
int4model.load_state_dict(torch.load('int4model.ckpt')())

bfp12model = deepcopy(model)
bfp12model.load_state_dict(torch.load('bfp12model.ckpt')())

<All keys matched successfully>

In [3]:
train,test = get_loader('cifar10'.upper(),batch_size=128,test_batch_size=512)
train.num_workers = 4
test.num_workers = 4

Files already downloaded and verified
Files already downloaded and verified


In [4]:
evaluate(test,bfp12model)

{'mean_acc': 0.9089,
 'qtl_acc': 0.9089,
 'mean_loss': 0.34937334805727005,
 'qtl_loss': 0.34937334805727005,
 'test time': 0.8863584995269775,
 'acc_list': array([0.9089]),
 'loss_list': array([0.34937335])}

In [5]:
int4params,_ = pyhessian.get_params_grad(int4model)
bfp12params,_ = pyhessian.get_params_grad(bfp12model)
fp32params,_ = pyhessian.get_params_grad(model)
int4_error = [(x-y) for x,y in zip(int4params,fp32params)]
bfp12_error = [(x-y) for x,y in zip(bfp12params,fp32params)]

In [6]:
int4_l2 = pyhessian.group_product(int4_error,int4_error)**0.5
bfp12_l2 = pyhessian.group_product(bfp12_error,bfp12_error)**0.5

In [7]:
int4_error_var = [x.std() for x in int4_error]
bfp12_error_var = [x.std() for x in bfp12_error]

In [8]:
for x,y in zip(int4_error_var,bfp12_error_var):
    print(x/int4_l2,y/bfp12_l2)
    print()

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0.0043, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0030, device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0.0036, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0030, device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0., device='cuda:0', grad_fn=<DivBackward0>) tensor(0., device='cuda:0', grad_fn=<DivBackward0>)

tensor(0.0027, device='cuda:0', grad_fn

In [9]:
def replaceModuleByName(modelName,moduleName,newModuleName):
    '''
        replace module with name modelName.moduleName with newModule
    '''
    tokens = moduleName.split('.')
    eval_str = modelName
    for token in tokens:
        try:
            eval_str += f'[{int(token)}]'
        except:
            eval_str += f'.{token}'
            
    exec(eval_str+f'={newModuleName}')
    
for name,module in model.named_modules():
    if isinstance(module,torch.nn.Conv2d):
        #print(name,' is a conv2d')
        newLayer = qConv2d(0,0,0,init_from=module).cuda()
        replaceModuleByName('model',name,'newLayer')
    elif isinstance(module,torch.nn.Linear):
        #print(name,' is a linear')
        newLayer = qLinear(0,0,init_from=module).cuda()
        replaceModuleByName('model',name,'newLayer')

In [ ]:
evaluate(test,model,noise_std=0.2,repeat=100,debug=True)

test 1/100 [20 batches 0.5547 seconds]:
loss 0.4870 acc 0.8762
test 2/100 [20 batches 0.5924 seconds]:
loss 0.4282 acc 0.8859
test 3/100 [20 batches 0.5666 seconds]:
loss 0.5003 acc 0.8734
test 4/100 [20 batches 0.5466 seconds]:
loss 0.7593 acc 0.8162
test 5/100 [20 batches 0.5696 seconds]:
loss 0.6431 acc 0.8366
test 6/100 [20 batches 0.6038 seconds]:
loss 0.6848 acc 0.8364
test 7/100 [20 batches 0.5801 seconds]:
loss 0.4612 acc 0.8830
test 8/100 [20 batches 0.6352 seconds]:
loss 0.5914 acc 0.8434
test 9/100 [20 batches 0.5738 seconds]:
loss 0.5673 acc 0.8528
test 10/100 [20 batches 0.5855 seconds]:
loss 0.4061 acc 0.8919
test 11/100 [20 batches 0.5670 seconds]:
loss 0.5334 acc 0.8684
test 12/100 [20 batches 0.5618 seconds]:
loss 0.6005 acc 0.8426
test 13/100 [20 batches 0.5532 seconds]:
loss 0.5191 acc 0.8614
test 14/100 [20 batches 0.5935 seconds]:
loss 0.4605 acc 0.8801
test 15/100 [20 batches 0.5766 seconds]:
loss 0.5660 acc 0.8645
test 16/100 [20 batches 0.5743 seconds]:
loss 0.4

In [ ]:
C = {}
C['epochs'] = 30
C['optimizer'] ='SGD'
C['lr'] = 1e-3
C['decay_ep'] = 10
C['decay_ratio'] = 0.1
C['device'] = 'cuda'
C['valPerEp'] = 1
C['noise_std'] = 0.2
C['valSample'] = 10
C['trial_name'] = 'qavat_train'

In [ ]:
model.train()
QAVAT_train(model,train,test,config=C,imgSize=32,imgFlat=False,
                lossfunc=torch.nn.CrossEntropyLoss(),printPerEpoch=1)